# Install and Import Required Libraries

In [1]:
# !pip install pymc arviz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az



WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


# Data Loading: Reading Excel Files from Multiple Sources

In [6]:
#  Economic Activity by Ethnicity (Pre-2021 and 2021)
df1_dict = pd.read_excel("./data/eco_eth.xlsx",sheet_name=None,skiprows=9)
df2 = pd.read_excel("./data_2021/eco_eth_2021.xlsx",sheet_name="Dataset")

#  Industry by Ethnicity (Pre-2021 and 2021)
df3_dict = pd.read_excel("./data/eth_ind.xlsx",sheet_name=None,skiprows=8 )
df4 = pd.read_excel("./data_2021/eth_ind_2021.xlsx",sheet_name="Dataset")

#  Occupation by Ethnicity (Pre-2021 and 2021)
df5_dict = pd.read_excel("./data/eth_occ.xlsx",sheet_name=None,skiprows=8)
df6 = pd.read_excel("./data_2021/eth_occ_2021.xlsx",sheet_name = "Dataset")



D:\Anaconda\envs\text_analytics\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\Anaconda\envs\text_analytics\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\Anaconda\envs\text_analytics\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


 ## Data Inspection: Displaying Sheet Names for All Datasets

 This code loops over all the sheets within each dictionary (df1_dict, df3_dict, df5_dict) to print the names of the sheets.

In [7]:
#  Inspecting Sheets from eco_eth.xlsx (Pre-2021)
for sheet_name, df in df1_dict.items():
    print(f"Sheet: {sheet_name}")
print("\n")

# Inspecting Sheet from eco_eth_2021.xlsx (2021)
for sheet_name, df in df2.items():
    print(f"Sheet: {sheet_name}")
print("\n")

# Inspecting Sheets from eth_ind.xlsx (Pre-2021)
for sheet_name, df in df3_dict.items():
    print(f"Sheet: {sheet_name}")
print("\n")

# Inspecting Sheets from eth_occ.xlsx (Pre-2021)
for sheet_name, df in df5_dict.items():
    print(f"Sheet: {sheet_name}")
print("\n")
  

Sheet: Economically active  In employm
Sheet: Sheet 1
Sheet: Economically active  Unemployed
Sheet: Sheet 2
Sheet: Economically inactive  Total
Sheet: Economically inactive  Student 


Sheet: Lower tier local authorities Code
Sheet: Lower tier local authorities
Sheet: Economic activity status (7 categories) Code
Sheet: Economic activity status (7 categories)
Sheet: Ethnic group (8 categories) Code
Sheet: Ethnic group (8 categories)
Sheet: Observation


Sheet: White  English or Welsh or Scot
Sheet: White  Irish
Sheet: White  Other White
Sheet: Mixed or multiple ethnic group
Sheet: Asian or Asian British
Sheet: Black or African or Caribbean o
Sheet: Other ethnic group


Sheet: White  English or Welsh or Scot
Sheet: White  Irish
Sheet: White  Other White
Sheet: Mixed or multiple ethnic group
Sheet: Asian or Asian British
Sheet: Black or African or Caribbean o
Sheet: Other ethnic group




## Defining Sheet Names and Labels for Ethnic and Age Groups

df1_sheets: This dictionary maps the sheet names in the eco_eth.xlsx file to the corresponding labels representing different economic groups (e.g., "Economically Active: Employed Students").

df3_sheets and df5_sheets: These dictionaries map sheet names in both the eth_ind.xlsx and eth_occ.xlsx files to ethnic group labels (e.g., "White: English/Welsh/Scottish/Northern Irish/British").

In [8]:
# Mapping of sheet names from eco_eth.xlsx to corresponding ethnic group and age group labels

df1_sheets = {

"Economically active  In employm":"Economically Active: In Employment: Total",
"Sheet 1":"Economically Active: Employed Students",
"Economically active  Unemployed":"Economically Active: Unemployed",
"Sheet 2":"Economically Active: Unemployed Students",
"Economically inactive  Total":"Economically inactive: Total",
"Economically inactive  Student ":"Economically inactive: Students"

}

In [9]:
# Mapping of sheet names from eth_ind.xlsx and eth_occ.xlsx to corresponding ethnic group labels
# This dictionary applies to both df3_dict and df5_dict, as they share the same structure

df3_sheets = df5_sheets = {
    "White  English or Welsh or Scot":'White: English/Welsh/Scottish/Northern Irish/British',
    "White  Irish":"White: Irish",
    "White  Other White": "White: Other White",
    "Mixed or multiple ethnic group":"Mixed/multiple ethnic group",
    "Asian or Asian British":"Asian/Asian British",
    "Black or African or Caribbean o":"Black/African/Caribbean/Black British",
    "Other ethnic group": "Other ethnic group"
}

# Data Cleaning and Combining

- Iterating Over Sheets: The loop goes through each sheet in df1_dict (which contains all the sheets from the eco_eth.xlsx file). It checks if the sheet name exists in the df1_sheets dictionary, which maps sheet names to their corresponding ethnic and economic activity labels.
- Cleaning Data:
   - Removing Meta Data: The code removes the last two rows (df.iloc[:-2]) that are considered meta data.
   - Resetting Index: After removing the rows, the DataFrame's index is reset using reset_index(drop=True) to avoid any gaps.
   - Adding Labels: It adds a new column, 'Economic Activity', based on the sheet’s label from the df1_sheets dictionary to classify each record.

- Combining Data: All the cleaned DataFrames are appended to a list and then combined into one large DataFrame (combined_df1) using pd.concat().
- Final Adjustments:
   - Column Cleaning: Strips any leading or trailing spaces in the column names (optional).
   - Renaming Columns: Renames some of the columns for clarity and consistency in the final DataFrame (combined_df1).

In [10]:
# Iterate Over Each Sheet and Clean Data
cleaned_df1 = []  # Create an empty list to store cleaned DataFrames

# Iterate over the dictionary of DataFrames
for sheet_name, df in df1_dict.items():
    # Check if the sheet_name exists in our ethnicity_age_groups
    if sheet_name in df1_sheets:
        economic_activity = df1_sheets[sheet_name]
        
        # Clean meta data (remove rows outside the desired range)
        df = df.iloc[:-2]  # Adjusting to clean out meta data
        
        # Reset index after cleaning (optional)
        df.reset_index(drop=True, inplace=True)
        # Add columns for ethnic group and age group
        df['Economic Activity'] = economic_activity
        
        # # Optional: Clean the column names (remove leading/trailing spaces, if any)
        # df.columns = [col.strip() for col in df.columns]
        
        # Append cleaned DataFrame to the list
        cleaned_df1.append(df)


# Combine All Cleaned DataFrames and Final Adjustments

combined_df1 = pd.concat(cleaned_df1, ignore_index=True)
combined_df1.columns = [col.strip() for col in combined_df1.columns]
combined_df1.rename(columns={combined_df1.columns[1]: 'Area Code',"local authority: district / unitary (prior to April 2015)":"Local Authority Area"}, inplace=True)



C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\2524378672.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Economic Activity'] = economic_activity
C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\2524378672.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Economic Activity'] = economic_activity
C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\2524378672.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [11]:
combined_df1.columns

Index(['Local Authority Area', 'Area Code',
       'White: English/Welsh/Scottish/Northern Irish/British', 'White: Irish',
       'White: Other White', 'Mixed/multiple ethnic group',
       'Asian/Asian British', 'Black/African/Caribbean/Black British',
       'Other ethnic group', 'Economic Activity'],
      dtype='object')

- Column Removal and Renaming: The columns that were deemed unnecessary or overly complex are removed, and remaining columns are renamed for better clarity.
- Data Filtering: Rows that had non-relevant values, such as "Does not apply" for "Ethnic Group" or "Economic Activity", are filtered out to ensure the data is meaningful.

In [12]:
# Drop Unwanted Columns
df2.drop(["Economic activity status (7 categories) Code","Ethnic group (8 categories) Code"],inplace=True,axis=1)

#  Rename Columns for Clarity
df2.rename(columns={"Economic activity status (7 categories)":"Economic Activity", "Ethnic group (8 categories)":"Ethnic Group","Lower tier local authorities Code":"Area Code","Lower tier local authorities":"Local Authority Area"},inplace=True)

#  Remove Rows Based on Conditions
df2 = df2[~((df2["Ethnic Group"] == "Does not apply") & (df2["Observation"] == 0))].reset_index(drop=True)
df2 = df2[df2["Economic Activity"] != "Does not apply"].reset_index(drop=True)

# nspect Unique Values and Data Distribution
print(df2["Economic Activity"].unique())
print(df2["Area Code"].nunique())

['Economically active (excluding full-time students): In employment'
 'Economically active (excluding full-time students): Unemployed: Seeking work or waiting to start a job already obtained: Available to start working within 2 weeks'
 'Economically active and a full-time student: In employment'
 'Economically active and a full-time student: Unemployed: Seeking work or waiting to start a job already obtained: Available to start working within 2 weeks'
 'Economically inactive (excluding full-time students)'
 'Economically inactive and a full-time student']
331


- Mapping Categories: The final step maps the more complicated economic activity categories to simpler, standardized labels using a predefined mapping.

In [13]:
# Map Economic Activity Categories to a New Format
economic_mapping = {
    'Economically active (excluding full-time students): In employment':'Economically Active: In Employment',
 'Economically active (excluding full-time students): Unemployed: Seeking work or waiting to start a job already obtained: Available to start working within 2 weeks': 'Economically Active: Unemployed',
 'Economically active and a full-time student: In employment':'Economically Active: Employed Students',
 'Economically active and a full-time student: Unemployed: Seeking work or waiting to start a job already obtained: Available to start working within 2 weeks':'Economically Active: Unemployed Students',
 'Economically inactive (excluding full-time students)':"Economically inactive",
 'Economically inactive and a full-time student': 'Economically inactive: Students'
       
}

# Apply the mapping to the "Economic Activity" column
df2['Economic Activity']=df2['Economic Activity'].map(economic_mapping)

In [14]:
combined_df1["Economic Activity"].unique()

array(['Economically Active: In Employment: Total',
       'Economically Active: Employed Students',
       'Economically Active: Unemployed',
       'Economically Active: Unemployed Students',
       'Economically inactive: Total', 'Economically inactive: Students'],
      dtype=object)

In [15]:
df2['Economic Activity'].unique()

array(['Economically Active: In Employment',
       'Economically Active: Unemployed',
       'Economically Active: Employed Students',
       'Economically Active: Unemployed Students',
       'Economically inactive', 'Economically inactive: Students'],
      dtype=object)

In [16]:
# Initialize List for Cleaned DataFrames
cleaned_df3 = []

# Iterate over the dictionary of DataFrames
for sheet_name, df in df3_dict.items():
    # Check if the sheet_name exists in our ethnicity_age_groups
    if sheet_name in df3_sheets:
        ethnicity = df3_sheets[sheet_name]
        
        # Clean meta data (remove rows outside the desired range)
        df = df.iloc[:-2]  # Adjusting to clean out meta data
        
        # Reset index after cleaning (optional)
        df.reset_index(drop=True, inplace=True)
        
        df['Ethnic Group'] = ethnicity
        
        # # Optional: Clean the column names (remove leading/trailing spaces, if any)
        # df.columns = [col.strip() for col in df.columns]
        
        # Append cleaned DataFrame to the list
        cleaned_df3.append(df)


# Combine all DataFrames into a single DataFrame
combined_df3 = pd.concat(cleaned_df3, ignore_index=True)

# Clean Column Names and Rename
combined_df3.columns = [col.strip() for col in combined_df3.columns]
combined_df3.rename(columns={combined_df3.columns[1]: 'Area Code'}, inplace=True)


C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\3191154321.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ethnic Group'] = ethnicity
C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\3191154321.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ethnic Group'] = ethnicity
C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\3191154321.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [17]:
# Drop Unnecessary Columns
df4.drop(["Industry (current) (9 categories) Code","Ethnic group (8 categories) Code"],inplace=True,axis=1)

#  Rename Columns for Clarity
df4.rename(columns={"Industry (current) (9 categories)":"Industry", "Ethnic group (8 categories)":"Ethnic Group"},inplace=True)

#  Remove Unwanted Rows (Ethnic Group = "Does not apply" and Observation = 0)
df4 = df4[~((df4["Ethnic Group"] == "Does not apply") & (df4["Observation"] == 0))].reset_index(drop=True)

# Remove Rows Where Industry = "Does not apply"
df4 = df4[df4["Industry"] != "Does not apply"].reset_index(drop=True)

# Display Cleaned Data
df4

,Lower tier local authorities Code,Lower tier local authorities,Industry,Ethnic Group,Observation
0,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water","Asian, Asian British or Asian Welsh",8
1,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water","Black, Black British, Black Welsh, Caribbean o...",2
2,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water",Mixed or Multiple ethnic groups,11
3,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water","White: English, Welsh, Scottish, Northern Iris...",1535
4,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water",White: Irish,3
...,...,...,...,...,...
18531,W06000024,Merthyr Tydfil,"R, S, T, U Other",Mixed or Multiple ethnic groups,7
18532,W06000024,Merthyr Tydfil,"R, S, T, U Other","White: English, Welsh, Scottish, Northern Iris...",754
18533,W06000024,Merthyr Tydfil,"R, S, T, U Other",White: Irish,0
18534,W06000024,Merthyr Tydfil,"R, S, T, U Other","White: Gypsy or Irish Traveller, Roma or Other...",27


In [18]:
# Initialize an Empty List to Hold Cleaned DataFrames
cleaned_df5 = []

# Iterate over the dictionary of DataFrames
for sheet_name, df in df5_dict.items():
    # Check if the sheet_name exists in our mapping dictionary (ethnicity_age_groups)
    if sheet_name in df5_sheets:
        ethnicity = df5_sheets[sheet_name]
        
        # Clean meta data (remove rows outside the desired range)
        df = df.iloc[:-2]  # Adjusting to clean out meta data
        
        # Reset index after cleaning (optional)
        df.reset_index(drop=True, inplace=True)

        # Add the Ethnic Group Column
        df['Ethnic Group'] = ethnicity
        
        # # Optional: Clean the column names (remove leading/trailing spaces, if any)
        # df.columns = [col.strip() for col in df.columns]
        
        # Append cleaned DataFrame to the list
        cleaned_df5.append(df)


# Combine all DataFrames into a single DataFrame
combined_df5 = pd.concat(cleaned_df5, ignore_index=True)

# Clean Column Names
combined_df5.columns = [col.strip() for col in combined_df5.columns]

# Rename the 'Area Code' Column
combined_df5.rename(columns={combined_df5.columns[1]: 'Area Code'}, inplace=True)


C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\572435401.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ethnic Group'] = ethnicity
C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\572435401.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ethnic Group'] = ethnicity
C:\Users\aman1\AppData\Local\Temp\ipykernel_134716\572435401.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [19]:
# Drop Unnecessary Columns
df6.drop(["Occupation (current) (10 categories) Code","Ethnic group (8 categories) Code"],inplace=True,axis=1)

# Rename Columns for Clarity
df6.rename(columns={"Occupation (current) (10 categories)":"Occupation", "Ethnic group (8 categories)":"Ethnic Group"},inplace=True)

# Remove Rows Where 'Ethnic Group' Is "Does not apply" and 'Observation' is 0
df6 = df6[~((df6["Ethnic Group"] == "Does not apply") & (df6["Observation"] == 0))].reset_index(drop=True)

# Remove Rows Where 'Occupation' is "Does not apply"
df6 = df6[df6["Occupation"] != "Does not apply"].reset_index(drop=True)

# Display the Cleaned DataFrame
df6

,Lower tier local authorities Code,Lower tier local authorities,Occupation,Ethnic Group,Observation
0,E06000001,Hartlepool,"1. Managers, directors and senior officials","Asian, Asian British or Asian Welsh",72
1,E06000001,Hartlepool,"1. Managers, directors and senior officials","Black, Black British, Black Welsh, Caribbean o...",4
2,E06000001,Hartlepool,"1. Managers, directors and senior officials",Mixed or Multiple ethnic groups,14
3,E06000001,Hartlepool,"1. Managers, directors and senior officials","White: English, Welsh, Scottish, Northern Iris...",3209
4,E06000001,Hartlepool,"1. Managers, directors and senior officials",White: Irish,5
...,...,...,...,...,...
20848,W06000024,Merthyr Tydfil,9. Elementary occupations,Mixed or Multiple ethnic groups,27
20849,W06000024,Merthyr Tydfil,9. Elementary occupations,"White: English, Welsh, Scottish, Northern Iris...",2903
20850,W06000024,Merthyr Tydfil,9. Elementary occupations,White: Irish,1
20851,W06000024,Merthyr Tydfil,9. Elementary occupations,"White: Gypsy or Irish Traveller, Roma or Other...",342


## Functions for Modifying Columns

Why use these functions?

- The two functions serve the purpose of adding a "Year" column and renaming certain columns to make the DataFrames more consistent and easier to work with.

- modify_cols is intended for DataFrames from 2011, whereas modify_cols_2021 is for those from 2021, with different column renaming conventions for each year.

In [20]:
# Function to Modify Columns for 2011 Data (modify_cols)
def modify_cols(df):
    df['Year'] = 2011
    df.rename(columns={df.columns[0]:"Local Authority Area"},inplace=True)
    return df

# Function to Modify Columns for 2021 Data (modify_cols_2021)
def modify_cols_2021(df):
    df['Year'] = 2021
    df.rename(columns={"Lower tier local authorities Code":"Area Code","Lower tier local authorities":"Local Authority Area"},inplace=True)
    return df

# Applying the Functions to DataFrames

#  Apply modify_cols for 2011 Data
combined_df1 = modify_cols(combined_df1)
combined_df3 = modify_cols(combined_df3)
combined_df5 = modify_cols(combined_df5)


# Apply modify_cols_2021 for 2021 Data
df2 = modify_cols_2021(df2)
df4 = modify_cols_2021(df4)
df6 = modify_cols_2021(df6)



## Pivoting the Data

Why Use This Function?

- Purpose: This function is used to transform DataFrames into a long format, making it easier to analyze and visualize data with many categorical variables (e.g., ethnic group, occupation, industry).
- Flexibility: The function is dynamic enough to pivot any column based on a static column, allowing for greater flexibility when working with different types of data.
- Outcome: After running the function, each of df1, df3, and df5 will contain a reshaped long-format DataFrame that can be further analyzed or plotted.

In [21]:
#  Function for Pivoting DataFrame (pivot_table)
def pivot_table(df,column,static_column):

    # Identify the columns that need to be pivoted (those that are not static columns)
    dim_val_col = [col for col in df.columns if col not in ['Local Authority Area', 'Area Code', "Year",static_column]]

    # Melt the DataFrame into a long format, pivoting the data around the given columns
    pivot_df = pd.melt(df, 
                     id_vars=["Area Code", 'Local Authority Area',"Year", static_column], 
                     value_vars=dim_val_col, 
                     var_name=column, 
                     value_name='Observation')

    return pivot_df

In [22]:
# Apply the pivot_table Function to combined_df1, combined_df3, and combined_df5


df1 = pivot_table(combined_df1,"Ethnic Group","Economic Activity")
df3 = pivot_table(combined_df3,"Industry","Ethnic Group")
df5 = pivot_table(combined_df5,"Occupation","Ethnic Group")

In [99]:
df1

,Area Code,Local Authority Area,Year,Economic Activity,Ethnic Group,Observation
0,E06000005,Darlington,2011,Economically Active: In Employment: Total,White: English/Welsh/Scottish/Northern Irish/B...,45957.0
1,E06000047,County Durham,2011,Economically Active: In Employment: Total,White: English/Welsh/Scottish/Northern Irish/B...,220929.0
2,E06000001,Hartlepool,2011,Economically Active: In Employment: Total,White: English/Welsh/Scottish/Northern Irish/B...,36597.0
3,E06000002,Middlesbrough,2011,Economically Active: In Employment: Total,White: English/Welsh/Scottish/Northern Irish/B...,48627.0
4,E06000057,Northumberland,2011,Economically Active: In Employment: Total,White: English/Welsh/Scottish/Northern Irish/B...,143304.0
...,...,...,...,...,...,...
14611,W06000018,Caerphilly,2011,Economically inactive: Students,Other ethnic group,18.0
14612,W06000019,Blaenau Gwent,2011,Economically inactive: Students,Other ethnic group,3.0
14613,W06000020,Torfaen,2011,Economically inactive: Students,Other ethnic group,4.0
14614,W06000021,Monmouthshire,2011,Economically inactive: Students,Other ethnic group,11.0


In [23]:
df1["Economic Activity"].unique()

array(['Economically Active: In Employment: Total',
       'Economically Active: Employed Students',
       'Economically Active: Unemployed',
       'Economically Active: Unemployed Students',
       'Economically inactive: Total', 'Economically inactive: Students'],
      dtype=object)

In [24]:
df2["Economic Activity"].unique()

array(['Economically Active: In Employment',
       'Economically Active: Unemployed',
       'Economically Active: Employed Students',
       'Economically Active: Unemployed Students',
       'Economically inactive', 'Economically inactive: Students'],
      dtype=object)

In [23]:
def replace_totals_with_non_student_adjusted(df):
    
# Adjusting "Economically Active: In Employment"
    active_subset = df[df["Economic Activity"].isin([
        "Economically Active: In Employment: Total",
        "Economically Active: Employed Students"
    ])]

    active_pivot = active_subset.pivot_table(
        index=["Area Code", "Local Authority Area", "Year", "Ethnic Group"],
        columns="Economic Activity",
        values="Observation",
        aggfunc="sum"
    )

    active_pivot["Observation"] = (
        active_pivot["Economically Active: In Employment: Total"] -
        active_pivot["Economically Active: Employed Students"]
    )

    active_result = active_pivot.reset_index()
    active_result["Economic Activity"] = "Economically Active: In Employment"
    active_result = active_result[[
        "Area Code", "Local Authority Area", "Year",
        "Economic Activity", "Ethnic Group", "Observation"
    ]]

# Adjusting "Economically Inactive" Categories
    inactive_subset = df[df["Economic Activity"].isin([
        "Economically inactive: Total",
        "Economically inactive: Students"
    ])]

    inactive_pivot = inactive_subset.pivot_table(
        index=["Area Code", "Local Authority Area", "Year", "Ethnic Group"],
        columns="Economic Activity",
        values="Observation",
        aggfunc="sum"
    )

    inactive_pivot["Observation"] = (
        inactive_pivot["Economically inactive: Total"] -
        inactive_pivot["Economically inactive: Students"]
    )

    inactive_result = inactive_pivot.reset_index()
    inactive_result["Economic Activity"] = "Economically inactive"
    inactive_result = inactive_result[[
        "Area Code", "Local Authority Area", "Year",
        "Economic Activity", "Ethnic Group", "Observation"
    ]]

# Combining Adjusted Data
    cleaned = df[
        ~df["Economic Activity"].isin([
            "Economically Active: In Employment: Total",
            "Economically inactive: Total"
        ])
    ]

    final_df = pd.concat([cleaned, active_result, inactive_result], ignore_index=True)

    # Return the Final DataFrame
    return final_df


In [24]:
# Adjust Economic Activity Data
combined_df = replace_totals_with_non_student_adjusted(df1)

In [25]:
df1 = combined_df.copy() # Save the adjusted DataFrame to df1

In [28]:
# Check Unique Values in Economic Activity Columns
combined_df1["Economic Activity"].unique()

array(['Economically Active: In Employment: Total',
       'Economically Active: Employed Students',
       'Economically Active: Unemployed',
       'Economically Active: Unemployed Students',
       'Economically inactive: Total', 'Economically inactive: Students'],
      dtype=object)

In [29]:
# Check Unique Values in Ethnic Group Columns
df1["Economic Activity"].unique()

array(['Economically Active: Employed Students',
       'Economically Active: Unemployed',
       'Economically Active: Unemployed Students',
       'Economically inactive: Students',
       'Economically Active: In Employment', 'Economically inactive'],
      dtype=object)

In [30]:
df2["Economic Activity"].unique()


array(['Economically Active: In Employment',
       'Economically Active: Unemployed',
       'Economically Active: Employed Students',
       'Economically Active: Unemployed Students',
       'Economically inactive', 'Economically inactive: Students'],
      dtype=object)

In [31]:
df1["Ethnic Group"].unique()


array(['White: English/Welsh/Scottish/Northern Irish/British',
       'White: Irish', 'White: Other White',
       'Mixed/multiple ethnic group', 'Asian/Asian British',
       'Black/African/Caribbean/Black British', 'Other ethnic group'],
      dtype=object)

In [32]:
df2["Ethnic Group"].unique()


array(['Asian, Asian British or Asian Welsh',
       'Black, Black British, Black Welsh, Caribbean or African',
       'Mixed or Multiple ethnic groups',
       'White: English, Welsh, Scottish, Northern Irish or British',
       'White: Irish',
       'White: Gypsy or Irish Traveller, Roma or Other White',
       'Other ethnic group'], dtype=object)

In [33]:
df3["Ethnic Group"].unique()


array(['White: English/Welsh/Scottish/Northern Irish/British',
       'White: Irish', 'White: Other White',
       'Mixed/multiple ethnic group', 'Asian/Asian British',
       'Black/African/Caribbean/Black British', 'Other ethnic group'],
      dtype=object)

In [34]:
df4["Ethnic Group"].unique()


array(['Asian, Asian British or Asian Welsh',
       'Black, Black British, Black Welsh, Caribbean or African',
       'Mixed or Multiple ethnic groups',
       'White: English, Welsh, Scottish, Northern Irish or British',
       'White: Irish',
       'White: Gypsy or Irish Traveller, Roma or Other White',
       'Other ethnic group'], dtype=object)

In [26]:
# Standardize Industry Category 
df4["Industry"] = df4["Industry"].replace(
    "K, L, M, N Financial, real estate, professional and administrative activities",
    "K, L, M, N Financial, Real Estate, Professional and Administrative activities"    
)

In [27]:
#  Ethnicity Mapping
ethnicity_mapping = {
    'Asian, Asian British or Asian Welsh':'Asian/Asian British',
       'Black, Black British, Black Welsh, Caribbean or African': 'Black/African/Caribbean/Black British',
       'Mixed or Multiple ethnic groups':'Mixed/multiple ethnic group',
       'White: English, Welsh, Scottish, Northern Irish or British':'White: English/Welsh/Scottish/Northern Irish/British',
       'White: Irish':'White: Irish',
       'White: Gypsy or Irish Traveller, Roma or Other White':'White: Other White',
       'Other ethnic group':'Other ethnic group'
}

#  Apply Ethnicity Mapping to DataFrames
df2['Ethnic Group'] = df2['Ethnic Group'].map(ethnicity_mapping)
df4['Ethnic Group'] = df4['Ethnic Group'].map(ethnicity_mapping)
df6['Ethnic Group'] = df6['Ethnic Group'].map(ethnicity_mapping)

Load and Prepare Area Mapping Data

In [61]:
# Load the area mapping CSV file which contains Local Authority District (LAD) codes for both 2011 and 2021
area_mappings = pd.read_csv("./data/Local_Authority_District_(2011)_to_Local_Authority_District_(2021)_Lookup_for_England_and_Wales.csv")

 # Extract Relevant Columns for Area Code Mapping
code_mapping = area_mappings[["LAD11CD", "LAD11NM","LAD21CD", "LAD21NM"]]
code_mapping.columns = ["Area Code 2011", "Area Name 2011","Area Code 2021", "Area Name 2021"]

# code_mapping[code_mapping["Area Code 2011"]!= code_mapping["Area Code 2021"]]

In [22]:
# Get unique 2011 and 2021 area codes
codes_2011 = set(df1["Area Code"].unique())
codes_2021 = set(df2["Area Code"].unique())

 # Identify Unmatched Area Codes in 2021
unmatched_2021_codes = list(codes_2021 - codes_2011)

# Extract Potentially Merged Districts
df_merged = df2[df2["Area Code"].isin(unmatched_2021_codes)]

In [70]:
# df_merged['Area Code'].unique()

In [23]:
#  Identify and Filter Area Code Mergers


# df1[df1['Area Code'] == "E06000029"]
# df2[df2['Area Code'] == "E06000058"]
# df2
merged_lookup = (                                       # Group the area mapping data by 'Area Code 2021' and filter those where there are more than one unique 'Area Code 2011'
    code_mapping.groupby("Area Code 2021")
    .filter(lambda g: len(g["Area Code 2011"].unique()) > 1)   # Identify merged areas by checking for multiple 2011 area codes
    .sort_values("Area Code 2021")    # Sort the results by the 2021 area code for easier viewing
)

# Output the Resulting Merged Lookup Data
merged_lookup

,Area Code 2011,Area Name 2011,Area Code 2021,Area Name 2021
54,E06000028,Bournemouth,E06000058,"Bournemouth, Christchurch and Poole"
55,E06000029,Poole,E06000058,"Bournemouth, Christchurch and Poole"
56,E07000048,Christchurch,E06000058,"Bournemouth, Christchurch and Poole"
293,E07000053,Weymouth and Portland,E06000059,Dorset
291,E07000051,Purbeck,E06000059,Dorset
290,E07000050,North Dorset,E06000059,Dorset
292,E07000052,West Dorset,E06000059,Dorset
57,E07000049,East Dorset,E06000059,Dorset
295,E07000005,Chiltern,E06000060,Buckinghamshire
294,E07000004,Aylesbury Vale,E06000060,Buckinghamshire


In [25]:
# Extract Merged 2021 Area Codes
merged_2021_codes = merged_lookup["Area Code 2021"].unique()

# Display the merged 2021 area codes
merged_2021_codes

array(['E06000058', 'E06000059', 'E06000060', 'E06000061', 'E06000062',
       'E07000244', 'E07000245', 'E07000246'], dtype=object)

In [26]:
# Filter 2021 Data for Merged Area Codes
merged_df2 = df2[df2["Area Code"].isin(merged_2021_codes)].copy()
merged_df4 = df4[df4["Area Code"].isin(merged_2021_codes)].copy()
merged_df6 = df6[df6["Area Code"].isin(merged_2021_codes)].copy()



## Bayesian Split for Merged Area Counts

This code block effectively splits the total counts for merged areas in 2021 into estimates for each of the corresponding 2011 areas using a Bayesian approach, and stores the results for further analysis.

In [27]:
def split_counts_bayesian(total_count, num_splits):
    with pm.Model():
        # Prior: assume equal probability for each split initially
        proportions = pm.Dirichlet('proportions', a=np.ones(num_splits))
        
        # Simulate actual counts from multinomial (based on Dirichlet proportions)
        split_counts = pm.Deterministic('split_counts', total_count * proportions)
        
        # Draw samples
        trace = pm.sample(200, tune=200, chains=2, progressbar=True, random_seed=42, return_inferencedata=True)
        
    # Take the mean predicted count from posterior
    return trace.posterior['split_counts'].mean(dim=("chain", "draw")).values


In [39]:
# Apply Bayesian Split for Merged Area Data
results = []

for _, row in merged_df2.iterrows():
    merged_code = row["Area Code"]
    total_count = row["Observation"]
    
    # Get corresponding 2011 area codes for this merged district
    corresponding_2011_codes = merged_lookup[merged_lookup["Area Code 2021"] == merged_code]["Area Code 2011"].unique()
    
    # Run Bayesian split
    split_values = split_counts_bayesian(total_count, len(corresponding_2011_codes))
    
    # Store one result per 2011 area
    for code_2011, est_count in zip(corresponding_2011_codes, split_values):
        results.append({
            "Area Code 2011": code_2011,
            "Area Name 2011": code_mapping[code_mapping["Area Code 2011"] == code_2011]["Area Name 2011"].values[0],
            "Area Code 2021": merged_code,
            "Area Name 2021": row["Local Authority Area"],
            "Ethnic Group": row["Ethnic Group"],
            "Economic Activity": row["Economic Activity"],
            "Estimated Count": int(round(est_count))
        })

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [40]:
# Save Results to CSV
split_df = pd.DataFrame(results)  # Convert the list of results into a DataFrame
split_df.to_csv("Bayesian_Eth_Eco21.csv") # Save the DataFrame as a CSV file for further use

In [74]:
# Load and Display the Saved CSV

# Read the CSV file back into a DataFrame
split_df = pd.read_csv("./data/Bayesian_Eth_Eco21.csv",index_col=0)

# Display the DataFrame to verify its contents
split_df

,Area Code 2011,Area Name 2011,Area Code 2021,Area Name 2021,Ethnic Group,Economic Activity,Estimated Count
0,E06000028,Bournemouth,E06000058,"Bournemouth, Christchurch and Poole",Asian/Asian British,Economically Active: In Employment,2024
1,E06000029,Poole,E06000058,"Bournemouth, Christchurch and Poole",Asian/Asian British,Economically Active: In Employment,2217
2,E07000048,Christchurch,E06000058,"Bournemouth, Christchurch and Poole",Asian/Asian British,Economically Active: In Employment,2047
3,E06000028,Bournemouth,E06000058,"Bournemouth, Christchurch and Poole",Black/African/Caribbean/Black British,Economically Active: In Employment,617
4,E06000029,Poole,E06000058,"Bournemouth, Christchurch and Poole",Black/African/Caribbean/Black British,Economically Active: In Employment,676
...,...,...,...,...,...,...,...
1045,E07000190,Taunton Deane,E07000246,Somerset West and Taunton,White: Irish,Economically inactive: Students,5
1046,E07000191,West Somerset,E07000246,Somerset West and Taunton,White: Other White,Economically inactive: Students,93
1047,E07000190,Taunton Deane,E07000246,Somerset West and Taunton,White: Other White,Economically inactive: Students,106
1048,E07000191,West Somerset,E07000246,Somerset West and Taunton,Other ethnic group,Economically inactive: Students,16


This block of code cleans up the original 2021 dataset by removing merged districts, renaming columns for consistency, and combining the original and split data into one final DataFrame (df2_final).

Sorting is an optional step that can improve the clarity of the final dataset.

In [41]:
# Identify Merged Area Codes
merged_codes = split_df['Area Code 2021'].unique()

# Remove Merged Districts from the Original 2021 Dataset
df2_cleaned = df2[~df2['Area Code'].isin(merged_codes)].copy()

# Rename Columns in Split Data
split_df_renamed = split_df.rename(columns={
    'Area Code 2011': 'Area Code',
    'Area Name 2011': 'Local Authority Area',
    'Estimated Count': 'Observation'
})

# Ensure Year column is set correctly
split_df_renamed['Year'] = 2021

# Concatenate Cleaned Data with Split Data
df2_final = pd.concat([df2_cleaned, split_df_renamed[df2.columns]], ignore_index=True)

# # Optional: sort for cleanliness
# df2_final = df_2021_final.sort_values(by=['Area Code', 'Economic Activity', 'Ethnic Group'])

In [42]:
# Saving the Final Dataset to a CSV File
df2_final.to_csv("Final_Eco_Eth_2021.csv")

In [35]:
# Define Result Storage List
results = []

# Iterate Over Merged 2021 Districts
for _, row in merged_df4.iterrows():
    merged_code = row["Area Code"]
    total_count = row["Observation"]
    
    # Get corresponding 2011 area codes for this merged district
    corresponding_2011_codes = merged_lookup[merged_lookup["Area Code 2021"] == merged_code]["Area Code 2011"].unique()
    
    # Run Bayesian split for Each Merged District
    split_values = split_counts_bayesian(total_count, len(corresponding_2011_codes))
    
    # Store Split Results for Each 2011 Area
    for code_2011, est_count in zip(corresponding_2011_codes, split_values):
        results.append({
            "Area Code 2011": code_2011,
            "Area Name 2011": code_mapping[code_mapping["Area Code 2011"] == code_2011]["Area Name 2011"].values[0],
            "Area Code 2021": merged_code,
            "Area Name 2021": row["Local Authority Area"],
            "Ethnic Group": row["Ethnic Group"],
            "Industry": row["Industry"],
            "Estimated Count": int(round(est_count))
        })

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [36]:
# Saving the Industry Split Results to CSV
split_df2 = pd.DataFrame(results)
split_df2.to_csv("Bayesian_Eth_Ind21.csv")

In [ ]:
split_df2

In [37]:
# Identify merged area codes (these were used for Bayesian splitting)
merged_codes = split_df2['Area Code 2021'].unique()

# Remove merged districts from the original 2021 dataset
df4_cleaned = df4[~df4['Area Code'].isin(merged_codes)].copy()

split_df_renamed = split_df2.rename(columns={
    'Area Code 2011': 'Area Code',
    'Area Name 2011': 'Local Authority Area',
    'Estimated Count': 'Observation'
})

# Ensure Year column is set correctly
split_df_renamed['Year'] = 2021

# Concatenate the cleaned df_2021 with the split data
df4_final = pd.concat([df4_cleaned, split_df_renamed[df4.columns]], ignore_index=True)

# Save Final Dataset to CSV
df4_final.to_csv("Final_Eth_Ind21.csv")

# # Optional: sort for cleanliness
# df2_final = df_2021_final.sort_values(by=['Area Code', 'Economic Activity', 'Ethnic Group'])

In [29]:
# Bayesian Redistribution for Merged Occupation Data (2021)

results = []

# Loop Through Merged Occupation Rows:
for _, row in merged_df6.iterrows():
    merged_code = row["Area Code"]
    total_count = row["Observation"]
    
    # Get corresponding 2011 area codes for this merged district
    corresponding_2011_codes = merged_lookup[merged_lookup["Area Code 2021"] == merged_code]["Area Code 2011"].unique()
    
    # Run Bayesian split
    split_values = split_counts_bayesian(total_count, len(corresponding_2011_codes))
    
    # Store one result per 2011 area
    for code_2011, est_count in zip(corresponding_2011_codes, split_values):
        results.append({
            "Area Code 2011": code_2011,
            "Area Name 2011": code_mapping[code_mapping["Area Code 2011"] == code_2011]["Area Name 2011"].values[0],
            "Area Code 2021": merged_code,
            "Area Name 2021": row["Local Authority Area"],
            "Ethnic Group": row["Ethnic Group"],
            "Occupation": row["Occupation"],
            "Estimated Count": int(round(est_count))
        })

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [33]:
# Exporting Bayesian Occupation Split Results to CSV
split_df3 = pd.DataFrame(results)
split_df3.to_csv("Bayesian_Eth_Occ21.csv")

In [31]:
# Identify merged area codes (these were used for Bayesian splitting)
merged_codes = split_df3['Area Code 2021'].unique()

# Remove merged districts from the original 2021 dataset
df6_cleaned = df6[~df6['Area Code'].isin(merged_codes)].copy()

split_df_renamed = split_df3.rename(columns={
    'Area Code 2011': 'Area Code',
    'Area Name 2011': 'Local Authority Area',
    'Estimated Count': 'Observation'
})

# Ensure Year column is set correctly
split_df_renamed['Year'] = 2021

# Concatenate the cleaned df_2021 with the split data
df6_final = pd.concat([df6_cleaned, split_df_renamed[df6.columns]], ignore_index=True)

# # Optional: sort for cleanliness
# df2_final = df_2021_final.sort_values(by=['Area Code', 'Economic Activity', 'Ethnic Group'])

In [34]:
df6_final.to_csv("Final_Eth_Occ21.csv")

- Purpose:
This code loads the finalized, cleaned, and merged 2021 datasets that include Bayesian-adjusted values for districts that changed between the 2011 and 2021 census boundaries.

- Details:
   - df2 → Final Economic Activity dataset (Ethnicity x Economic Status).
   - df4 → Final Industry dataset (Ethnicity x Industry).
   - df6 → Final Occupation dataset (Ethnicity x Occupation).

- Note on index_col=0:
This treats the first column in the CSV (usually an automatically saved index) as the index of the DataFrame, keeping the structure clean.



In [32]:
# Loading Final Cleaned and Merged 2021 Datasets
df2 = pd.read_csv("./data/Final_Eco_Eth_2021.csv",index_col=0)
df4 = pd.read_csv("./data/Final_Eth_Ind21.csv",index_col=0)
df6 = pd.read_csv("./data/Final_Eth_Occ21.csv",index_col=0)


In [33]:
df2

,Area Code,Local Authority Area,Economic Activity,Ethnic Group,Observation,Year
0,E06000001,Hartlepool,Economically Active: In Employment,Asian/Asian British,638,2021
1,E06000001,Hartlepool,Economically Active: In Employment,Black/African/Caribbean/Black British,113,2021
2,E06000001,Hartlepool,Economically Active: In Employment,Mixed/multiple ethnic group,167,2021
3,E06000001,Hartlepool,Economically Active: In Employment,White: English/Welsh/Scottish/Northern Irish/B...,35155,2021
4,E06000001,Hartlepool,Economically Active: In Employment,White: Irish,71,2021
...,...,...,...,...,...,...
14611,E07000190,Taunton Deane,Economically inactive: Students,White: Irish,5,2021
14612,E07000191,West Somerset,Economically inactive: Students,White: Other White,93,2021
14613,E07000190,Taunton Deane,Economically inactive: Students,White: Other White,106,2021
14614,E07000191,West Somerset,Economically inactive: Students,Other ethnic group,16,2021


In [46]:
# Checking DataFrames After Bayesian Adjustment
df2_final['Area Code']

348

In [44]:
df4_final

,Area Code,Local Authority Area,Industry,Ethnic Group,Observation,Year
0,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water",Asian/Asian British,8,2021
1,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water",Black/African/Caribbean/Black British,2,2021
2,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water",Mixed/multiple ethnic group,11,2021
3,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water",White: English/Welsh/Scottish/Northern Irish/B...,1535,2021
4,E06000001,Hartlepool,"A, B, D, E Agriculture, energy and water",White: Irish,3,2021
...,...,...,...,...,...,...
19483,E07000190,Taunton Deane,"R, S, T, U Other",White: Irish,7,2021
19484,E07000191,West Somerset,"R, S, T, U Other",White: Other White,72,2021
19485,E07000190,Taunton Deane,"R, S, T, U Other",White: Other White,82,2021
19486,E07000191,West Somerset,"R, S, T, U Other",Other ethnic group,7,2021


In [45]:
df6_final

,Area Code,Local Authority Area,Occupation,Ethnic Group,Observation,Year
0,E06000001,Hartlepool,"1. Managers, directors and senior officials",Asian/Asian British,72,2021
1,E06000001,Hartlepool,"1. Managers, directors and senior officials",Black/African/Caribbean/Black British,4,2021
2,E06000001,Hartlepool,"1. Managers, directors and senior officials",Mixed/multiple ethnic group,14,2021
3,E06000001,Hartlepool,"1. Managers, directors and senior officials",White: English/Welsh/Scottish/Northern Irish/B...,3209,2021
4,E06000001,Hartlepool,"1. Managers, directors and senior officials",White: Irish,5,2021
...,...,...,...,...,...,...
21919,E07000190,Taunton Deane,9. Elementary occupations,White: Irish,14,2021
21920,E07000191,West Somerset,9. Elementary occupations,White: Other White,353,2021
21921,E07000190,Taunton Deane,9. Elementary occupations,White: Other White,400,2021
21922,E07000191,West Somerset,9. Elementary occupations,Other ethnic group,29,2021


In [43]:
df3.columns

Index(['Area Code', 'Local Authority Area', 'Year', 'Industry', 'Ethnic Group',
       'Observation'],
      dtype='object')

In [85]:
set(df1['Local Authority Area']) - set(df2['Local Authority Area'])

{'Bristol, City of',
 'Herefordshire, County of',
 'Kingston upon Hull, City of',
 'Rhondda Cynon Taff'}

In [86]:
# Mapping Area Codes to Local Authority Areas (2011 Data)
area_name_map_2011 = df1[['Area Code', 'Local Authority Area']].drop_duplicates().set_index('Area Code')['Local Authority Area']

In [88]:
# Mapping for df2
df2['Local Authority Area'] = df2['Area Code'].map(area_name_map_2011)

# Mapping for df4
df4['Local Authority Area'] = df4['Area Code'].map(area_name_map_2011)

# Mapping for df6
df6['Local Authority Area'] = df6['Area Code'].map(area_name_map_2011)

In [89]:
# Aligning df2 with the columns of df1
df2 = df2[df1.columns]

# Aligning df4 with the columns of df3
df4 = df4[df3.columns]

# Aligning df6 with the columns of df5
df6 = df6[df5.columns]

In [90]:
# Deleting com_df1
del com_df1

# Deleting com_df2
del com_df2

# Deleting com_df3
del com_df3


In [91]:
# Concatenating DataFrames to Create Combined DataFrames

# Combining df1 and df2 into com_df1
com_df1 = pd.concat([df1, df2], ignore_index=True)

# Combining df3 and df4 into com_df2
com_df2 = pd.concat([df3, df4], ignore_index=True)

# Combining df5 and df6 into com_df3
com_df3 = pd.concat([df5, df6], ignore_index=True)


In [53]:
# Checking Unique Values in the 'Ethnic Group' Column of com_df2
com_df2['Ethnic Group'].unique()

array(['White: English/Welsh/Scottish/Northern Irish/British',
       'White: Irish', 'White: Other White',
       'Mixed/multiple ethnic group', 'Asian/Asian British',
       'Black/African/Caribbean/Black British', 'Other ethnic group'],
      dtype=object)

In [54]:
# Checking Unique Values in the 'Ethnic Group' Column of com_df3
com_df3["Occupation"].unique()

array(['1. Managers, directors and senior officials',
       '2. Professional occupations',
       '3. Associate professional and technical occupations',
       '4. Administrative and secretarial occupations',
       '5. Skilled trades occupations',
       '6. Caring, leisure and other service occupations',
       '7. Sales and customer service occupations',
       '8. Process, plant and machine operatives',
       '9. Elementary occupations'], dtype=object)

In [51]:
# Analyzing Differences and Saving DataFrames

# Finding Local Authority Areas in com_df1 not present in df1
set(com_df1["Local Authority Area"]) - set(df1["Local Authority Area"])

{'Bristol', 'Herefordshire', 'Kingston upon Hull', 'Rhondda Cynon Taf'}

In [40]:
# Getting unique values in the 'Local Authority Area' column of com_df2
com_df2['Local Authority Area'].unique()

array(['Darlington', 'County Durham', 'Hartlepool', 'Middlesbrough',
       'Northumberland', 'Redcar and Cleveland', 'Stockton-on-Tees',
       'Gateshead', 'Newcastle upon Tyne', 'North Tyneside',
       'South Tyneside', 'Sunderland', 'Blackburn with Darwen',
       'Blackpool', 'Cheshire East', 'Cheshire West and Chester',
       'Halton', 'Warrington', 'Allerdale', 'Barrow-in-Furness',
       'Carlisle', 'Copeland', 'Eden', 'South Lakeland', 'Bolton', 'Bury',
       'Manchester', 'Oldham', 'Rochdale', 'Salford', 'Stockport',
       'Tameside', 'Trafford', 'Wigan', 'Burnley', 'Chorley', 'Fylde',
       'Hyndburn', 'Lancaster', 'Pendle', 'Preston', 'Ribble Valley',
       'Rossendale', 'South Ribble', 'West Lancashire', 'Wyre',
       'Knowsley', 'Liverpool', 'Sefton', 'St. Helens', 'Wirral',
       'East Riding of Yorkshire', 'Kingston upon Hull, City of',
       'North East Lincolnshire', 'North Lincolnshire', 'York', 'Craven',
       'Hambleton', 'Harrogate', 'Richmondshire', 'Ry

In [99]:
# Saving the DataFrames to CSV files

com_df1.to_csv("./data/Economic_Ethnic.csv",index=0)
com_df2.to_csv("./data/Industry_Ethnic.csv",index=0)
com_df3.to_csv("./data/Occupation_Ethnic.csv",index=0)

In [93]:
# Getting the number of unique 'Local Authority Area' in com_df1
com_df1['Local Authority Area'].nunique()

348

# Dimensionality Reduction

- Pivoting the DataFrame: The pivot_table() function is used to reshape the DataFrame com_df3 by setting 'Area Code', 'Local Authority Area', 'Ethnic Group', and 'Year' as the index. The columns are then created based on the different 'Occupation' categories, and the values are aggregated using the sum of 'Observation'.

- Handling Missing Data: If there are missing values in the data, you can use the fill_value parameter to fill them (e.g., with 0). A comment suggests using Bayesian imputation if needed.

- Index Reset: The reset_index() ensures that the pivoted data returns to a standard DataFrame format, as the original index columns will become regular columns.

In [95]:
# Importing Required Libraries for Data Preprocessing and Dimensionality Reduction
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Prepare the Data Matrix for t-SNE Analysis
# Pivot the data to wide format with occupations as columns
pivot_df = com_df3.pivot_table(
    index=['Area Code', 'Local Authority Area', 'Ethnic Group', 'Year'],
    columns='Occupation', 
    values='Observation',
    aggfunc='sum',
    # fill_value=0  # Use Bayesian imputation here if missing values exist
).reset_index()



In [885]:
# Displaying the resulting pivoted DataFrame
pivot_df

Occupation,Area Code,Local Authority Area,Ethnic Group,Year,"1. Managers, directors and senior officials",2. Professional occupations,3. Associate professional and technical occupations,4. Administrative and secretarial occupations,5. Skilled trades occupations,"6. Caring, leisure and other service occupations",7. Sales and customer service occupations,"8. Process, plant and machine operatives",9. Elementary occupations
0,E06000001,Hartlepool,Asian/Asian British,2011,83.0,145.0,23.0,25.0,78.0,40.0,50.0,23.0,61.0
1,E06000001,Hartlepool,Asian/Asian British,2021,72.0,178.0,49.0,26.0,79.0,53.0,105.0,36.0,62.0
2,E06000001,Hartlepool,Black/African/Caribbean/Black British,2011,2.0,21.0,6.0,4.0,5.0,9.0,4.0,3.0,7.0
3,E06000001,Hartlepool,Black/African/Caribbean/Black British,2021,4.0,25.0,15.0,3.0,10.0,24.0,7.0,15.0,14.0
4,E06000001,Hartlepool,Mixed/multiple ethnic group,2011,12.0,19.0,19.0,7.0,21.0,12.0,16.0,8.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4629,W06000024,Merthyr Tydfil,White: English/Welsh/Scottish/Northern Irish/B...,2021,2022.0,3402.0,2702.0,2145.0,2925.0,3049.0,2325.0,2126.0,2903.0
4630,W06000024,Merthyr Tydfil,White: Irish,2011,5.0,12.0,6.0,2.0,6.0,6.0,2.0,5.0,6.0
4631,W06000024,Merthyr Tydfil,White: Irish,2021,3.0,12.0,8.0,3.0,7.0,3.0,3.0,5.0,1.0
4632,W06000024,Merthyr Tydfil,White: Other White,2011,34.0,54.0,33.0,38.0,99.0,29.0,38.0,268.0,377.0


## TSNE: Dimensionality Reduction for Visualizing Occupations


In [96]:
## TSNE

# Standardize the Data
# Extract all occupation columns (columns starting from the 5th index onward)
occupation_cols = pivot_df.columns[4:]  # Get all occupation columns

# Initialize the StandardScaler to standardize the data
scaler = StandardScaler()

# Apply scaling to the occupation columns (standardizing the data)
scaled_occupations = scaler.fit_transform(pivot_df[occupation_cols])


# Apply t-SNE (with adjusted perplexity for smaller dataset)
# Initialize the t-SNE model with 2 components for 2D visualization, set random seed for reproducibility
tsne = TSNE(n_components=2, random_state=42, perplexity=30)

# Fit the t-SNE model and transform the scaled occupation data into 2D
tsne_results = tsne.fit_transform(scaled_occupations)

# Add the t-SNE results to the pivot_df
pivot_df['tSNE1'] = tsne_results[:, 0]
pivot_df['tSNE2'] = tsne_results[:, 1]

In [100]:
len(tsne_results)

# Save the data for Tableau (includes geographic and demographic context)
# Saving the DataFrame with t-SNE results for further visualization in Tableau

pivot_df.to_csv('./data/occupation_projections_tsne.csv', index=False)


## PCA: Principal Component Analysis for Visualizing Industry Data


In [98]:
from sklearn.decomposition import PCA

# Prepare the Data Matrix
# Pivot the data to wide format with industries as columns
pivot_df2 = com_df2.pivot_table(
    index=['Area Code', 'Local Authority Area', 'Ethnic Group', 'Year'],
    columns='Industry', 
    values='Observation',
    aggfunc='sum',
    # fill_value=0  # Use Bayesian imputation here if missing values exist
).reset_index()   # Reset the index to make it a standard DataFrame



# Standardize the Data
# Extract all industry columns (columns starting from the 5th index onward)
industry_cols = pivot_df2.columns[4:]  # Get all occupation columns

# Initialize the StandardScaler to standardize the data
scaler = StandardScaler()

# Apply scaling to the industry columns (standardizing the data)
scaled_industry = scaler.fit_transform(pivot_df2[industry_cols])

# Apply PCA (Principal Component Analysis)
# Initialize the PCA model with 2 components for 2D visualization
pca = PCA(n_components=2)

# Fit the PCA model and transform the scaled industry data into 2D
pca_results = pca.fit_transform(scaled_industry)

# Add the PCA results to the pivot_df2
pivot_df2['PCA1'] = pca_results[:, 0]
pivot_df2['PCA2'] = pca_results[:, 1]

In [101]:
# Save the data for Tableau (includes geographic and demographic context)
# Saving the DataFrame with PCA results for further visualization in Tableau
pivot_df2.to_csv('./data/industry_projections_pca.csv', index=False)
